In [8]:
import os
import csv
import json
from bs4 import BeautifulSoup
import shutil
import requests
import pandas as pd

In [30]:
recipes=[]

In [2]:
def get_cooking_time(soup):
    duration=""
    facts=soup.find_all('div',{'class':'facts'})
    for fact in facts:
        time=fact.find('div',{'class':'facts__item'})
        time1=time.find('dd',{'class':'facts__value'})
        duration=duration+time1.get_text()
    return duration

In [5]:
def get_ingredients(soup):
    ingredient=soup.find_all('ul',{'class':'ingredient-list'})
    ingredients=""
    for i in ingredient:
        ingri=i.find_all('li')
        for li in ingri:
            if(li.find('h4')):
                continue
            for span in li.find_all('span',{'class':'ingredient-text'}):
                ingredients=ingredients+span.get_text()+' '
            for quantity in li.find_all('span',{'class':'ingredient-quantity'}):
                ingredients=ingredients+quantity.get_text()+' '
    return ingredients

In [21]:
def get_directions(soup):
    directions=""
    ul=soup.find('ul',{'class':'direction-list'})
    li=ul.find_all('li',{'class':'direction'})
    for direction in li:
        directions=directions+direction.get_text()+"\n"
    return directions
        
        

In [25]:
def get_servings(soup):
    servings=""
    facts=soup.find_all('div',{'class':'facts'})
    for fact in facts:
        items=fact.find_all('div',{'class':'facts__item'})
        for item in items:
            serving=item.find('dd',{'class':'facts__value'})
            if(serving.find('span',{'class':'value'})):
                servings=servings+serving.find('span',{'class':'value'}).get_text()
            else:
                continue
    return servings

In [17]:
def get_calories(soup):
    calories = soup.find('script', type='application/ld+json').string
    calories = json.loads(calories)["nutrition"]["calories"]
    return calories

In [14]:
def text_scrape(recipe_name,url):
    data=requests.get(url)
    soup=BeautifulSoup(data.content,'html.parser')
    dicto={'name':recipe_name,'cooking_time':get_cooking_time(soup),'calories':get_calories(soup),'ingredients':get_ingredients(soup),'directions':get_directions(soup),'serving':get_servings(soup)}
    return dicto

In [34]:
def main():
    df=pd.read_csv("C:\\Users\\aditi\\OneDrive\\Desktop\\PROJECTS\\DEEP-CHEF-PROJECT\\ADITI\\recipes.csv")
    for index, row in df.iloc[0:2].iterrows():
        recipes.append(text_scrape(row['name'],row['link']))
    with open('recipes.json','r+') as f:
        json.dump(recipes,f)

if __name__=="__main__":
    main()